# TP2 Cryptographie et Sécurité

TANG Kévin p1501263

 Imaginons 2 questions publiques Q0, Q1. Alice a deux réponses secrètes r0, r1. Bob est intéressé par la réponse i ∈ {0, 1} mais ne souhaite pas divulguer cette information (i est secret). Peut-on construire un protocole T I qui garantit que :
- i et r1−i restent secrets.
- Bob connait ri à la fin du protocole.

Autrement dit, Bob peut-il obtenir une réponse à une question qu’il souhaite garder secrète ? Nous proposons le protocole (incomplet) suivant :

Transfert Inconscient (TI) :
1. Bob génère (pk, sk) ← P aillier.KeyGen(λ), publie pk et envoie I ← P aillier.
Encrypt(pk, i) à Alice.
2. Alice génère deux nombres aléatoires α0, α1 ∈ Z ∗ n et A0 ← P aillier.Encrypt (pk, 0) et A1 ← P aillier.Encrypt(pk, −1). Elle génère ensuite M0 = α0 ◦ (I ⊕ A0) et M1 = α1 ◦ (I ⊕ A1) puis R0 = . . . et R1 = . . . et envoie R0, R1 à Bob.
3. Bob retourne ri ← P aillier.Decrypt(sk, Ri).

**1. Completer le protocole TI afin qu’il soit correct.**

1. Bob génère (pk, sk) ← P aillier.KeyGen(λ), publie pk et envoie I ← P aillier.
Encrypt(pk, i) à Alice.
2. Alice génère deux nombres aléatoires α0, α1 ∈ Z ∗ n et A0 ← P aillier.Encrypt (pk, 0) et A1 ← P aillier.Encrypt(pk, −1). Elle génère ensuite M0 = α0 ◦ (I ⊕ A0) et M1 = α1 ◦ (I ⊕ A1) puis R0 = Paillier.Encrypt(pk, M0) et R1 = Paillier.Encrypt(pk, M1) et envoie R0, R1 à Bob.
3. Bob retourne ri ← P aillier.Decrypt(sk, Ri).

**2. Analyser ce protocole.**

À l'étape 1, Bob génère une paire de clés (pk, sk) pour le cryptosystème de Paillier et envoie une question i cryptée à Alice.

À l'étape 2, Alice génère deux nombres aléatoires α0 et α1 et utilise la question i pour calculer A0 et A1. Elle calcule ensuite M0 et M1 en combinant α0 et α1 avec I et A0 ou A1 respectivement. Elle chiffre M0 et M1 avec le cryptosystème de Paillier pour obtenir R0 et R1 qu'elle envoie à Bob.

À l'étape 3, Bob déchiffre R0 et R1 avec sa clé secrète sk pour obtenir M0 et M1. Il peut ensuite calculer r0 et r1 en déchiffrant A0 et A1 avec sa clé secrète sk. Il retourne ri correspondant à la réponse secrète associée à la question secrète i.

**3. Implémenter le protocole TI. Pour cela, on créera 3 méthodes TI1, TI2 et TI3 correspondant respectivement aux étapes 1,2 et 3 du protocole. TIk prendra en entrée que ce qui est connu par la partie exécutante (Alice ou Bob) au début de l’étape k. Pour simplifier l’implémentation, ”envoyer à” sera juste remplacé par ”retourner”. Par exemple, TI1 pourra prendre en entrée pk, sk, i et retournera une encryption de i.**

In [ ]:
import random
from Crypto.Util import number
from Crypto.PublicKey import Paillier
from Crypto.Cipher import PKCS1_OAEP

def TI1():
    # Step 1: Bob generates (pk, sk) and sends encrypted i to Alice
    key_length = 1024
    key = Paillier.generate(key_length)
    pk = key.publickey()
    sk = key
    i = random.randint(0, 1)
    cipher = PKCS1_OAEP.new(pk)
    I = cipher.encrypt(str(i).encode())
    return pk, sk, i, I

def TI2(pk, i, I):
    # Step 2: Alice generates random numbers and encrypts them, then sends them to Bob
    n = pk.n
    alpha0 = number.getRandomRange(1, n)
    alpha1 = number.getRandomRange(1, n)
    A0 = pow(pk.g, 0, n) * pow(pk.n + 1, alpha0, n) % n
    A1 = pow(pk.g, -1, n) * pow(pk.n + 1, alpha1, n) % n
    M0 = alpha0 * ((int.from_bytes(I, 'big') ^ A0) % n) % n
    M1 = alpha1 * ((int.from_bytes(I, 'big') ^ A1) % n) % n
    R0 = pow(M0, sk.l, n*n) * pow(A0, 1, n*n) % (n*n)
    R1 = pow(M1, sk.l, n*n) * pow(A1, 1, n*n) % (n*n)
    return R0, R1

def TI3(sk, R):
    # Step 3: Bob decrypts the result and gets the value of i
    n = sk.p * sk.q
    d = sk.invp
    plain = pow(R, d, n*n)
    M = (plain % n) % n
    I = PKCS1_OAEP.new(None).decrypt(M.to_bytes((M.bit_length() + 7) // 8, byteorder='big'))
    i = int.from_bytes(I, 'big')
    return sk.decrypt(plain), i

**4. Etendre ce protocole à un nombre quelconque de questions.**

Pour étendre le protocole TI à un nombre quelconque de questions, on peut utiliser le même principe que dans le protocole précédent en itérant simplement les étapes 2 et 3 autant de fois que nécessaire pour chaque question.

**5. Dire en quoi ce protocole peut être vu comme un protocole permettant de faire des requêtes sur une BD distante. Que garantit-il ?**

Le protocole TI peut être vu comme un protocole permettant de faire des requêtes sur une BD distante car il permet à une partie (Bob) de poser des questions (Q0, Q1, Q2, ...) à une autre partie (Alice) sans révéler les questions posées à une tierce partie. Alice peut être vue comme la BD distante, et Bob comme le client qui envoie des requêtes à la BD.

Ainsi, le protocole permet de réaliser des requêtes confidentielles sur une BD distante.